In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pickle
import csv

In [5]:
########################################################################################
#Unpickling the saved pass data of solar_envelope and solar_access
with open("data/solar_envelope_pass_1.txt", "rb") as fp:   # Unpickling
    envelope_pass_1 = pickle.load(fp)
    
with open("data/solar_envelope_pass_2.txt", "rb") as fp:   # Unpickling
    envelope_pass_2 = pickle.load(fp)
    
with open("data/solar_envelope_pass_3.txt", "rb") as fp:   # Unpickling
    envelope_pass_3 = pickle.load(fp)
    
    
    
with open("data/solar_access_pass_1.txt", "rb") as fp:   # Unpickling
    access_pass_1 = pickle.load(fp)
    
with open("data/solar_access_pass_2.txt", "rb") as fp:   # Unpickling
    access_pass_2 = pickle.load(fp)
    
with open("data/solar_access_pass_3.txt", "rb") as fp:   # Unpickling
    access_pass_3 = pickle.load(fp)
########################################################################################


########################################################################################
#Creating base lists for later reference 
#NOTE: this is a manually edited 14_4 file to hold the size as 3.6 in order to change that correctly
lattice_path = os.path.relpath("data\lattice_14_4_base_3_6.csv")
envelope_lattice = tg.lattice_from_csv(lattice_path)
envelope_enhanced = np.kron(envelope_lattice, np.ones((4, 4, 4), dtype=bool))
########################################################################################


########################################################################################
#Simple function for combining value data of two lattices
def CombineData(list1, list2):
    coords_list = []
    for i in range(len(list2)):
        for j in range(len(list2[0])):
            for k in range(len(list2[0][0])):
                if list2[i][j][k] != 2:
                    coords_list.append([i, j, k, list2[i][j][k]])
                    
    for i in coords_list:
        x = i[0]
        y = i[1]
        z = i[2]
        list1[i[0]][i[1]][i[2]] = i[3]
        
    return list1

#Simple function to set any 2 in a list to a 0.(Non existant voxels were saved with val=2)
def TwotoZero(input_list):
    output_list = input_list[:]
    for i in range(len(input_list)):
        for j in range(len(input_list[0])):
            for k in range(len(input_list[0][0])):
                if input_list[i][j][k] == 2:
                    output_list[i][j][k] = 0
    return output_list
########################################################################################


########################################################################################
#Combining the solar_envelope and solar_access steps
envelope_step_1 = CombineData(envelope_pass_1, envelope_pass_2)
envelope_step_2 = CombineData(envelope_step_1, envelope_pass_3)
envelope_final = TwotoZero(envelope_step_2)

access_step_1 = CombineData(access_pass_1, access_pass_2)
access_step_2 = CombineData(access_step_1, access_pass_3)
access_final = TwotoZero(access_step_2)
########################################################################################


########################################################################################
#Exporting combined lattices
#Pickle for easier later use
with open("data/solar_access_complete_3_6.txt", "wb") as fp:   #Pickling
    pickle.dump(access_final, fp)

#Generate CSV as official files
envelope_lattice = tg.to_lattice(envelope_final, envelope_enhanced)
access_lattice = tg.to_lattice(access_final, envelope_enhanced)

csv_path_envelope = os.path.relpath("data\solar_envelope_complete_3_6.csv")
csv_path_access = os.path.relpath("data\solar_access_complete_3_6.csv")

envelope_lattice.to_csv(csv_path_envelope)
access_lattice.to_csv(csv_path_access)
########################################################################################
print("done")
     

In [34]:
########################################################################################
#A simple definition of adding pass lattices to a base list
def CombineLattice(base_list, pass_list):
    for i in range(len(pass_list)):
        for j in range(len(pass_list[0])):
            for k in range(len(pass_list[0][0])):
                if pass_list[i][j][k] == True:
                    base_list[i][j][k] = True
    return base_list
########################################################################################


########################################################################################
#Establishing Base list filled with False
envelope_base = envelope_enhanced[:]
for i in range(len(envelope_enhanced)):
    for j in range(len(envelope_enhanced[0])):
        for k in range(len(envelope_enhanced[0][0])):
            envelope_base[i][j][k] = False
########################################################################################


########################################################################################
#Exporting the final combined lattice
lattice_pass_1_path = os.path.relpath("data\lattice_print_pass_1.csv")
lattice_pass_2_path = os.path.relpath("data\lattice_print_pass_2.csv")
lattice_pass_3_path = os.path.relpath("data\lattice_print_pass_3.csv")

lattice_pass_1 = tg.lattice_from_csv(lattice_pass_1_path)
lattice_pass_2 = tg.lattice_from_csv(lattice_pass_2_path)
lattice_pass_3 = tg.lattice_from_csv(lattice_pass_3_path)

lattice_step_1 = CombineLattice(envelope_base, lattice_pass_1)
lattice_step_2 = CombineLattice(lattice_step_1, lattice_pass_2)
lattice_final = CombineLattice(lattice_step_2, lattice_pass_3)

lattice_lattice = tg.to_lattice(lattice_final, envelope_enhanced)
csv_path_lattice = os.path.relpath("data\lattice_complete_3_6.csv")
lattice_lattice.to_csv(csv_path_lattice)
########################################################################################
print("done")